Przed oddaniem zadania upewnij się, że wszystko działa poprawnie.
**Uruchom ponownie kernel** (z paska menu: Kernel$\rightarrow$Restart) a następnie
**wykonaj wszystkie komórki** (z paska menu: Cell$\rightarrow$Run All).

Upewnij się, że wypełniłeś wszystkie pola `TU WPISZ KOD` lub `TU WPISZ ODPOWIEDŹ`, oraz
że podałeś swoje imię i nazwisko poniżej:

In [1]:
NAME = "Piotr Szuba"

---

# 2. Zadania w przetwarzaniu grafów

## 2.1. Uczenie reprezentacji wierzchołków

W poniższych rozważaniach będziemy się skupiać na zadaniu uczenia reprezentacji dla wierzchołków w grafie (ang. *node embedding*). Celem będzie znalezienie funkcji $h: \mathcal{V} \to \mathbb{R}^d$, która dla każdego wierzchołka $u$ wyznaczy (obliczy) jego wektor reprezentacji $\mathbf{z}_u$. Zobaczymy, że za pomocą reprezentacji wierzchołków możemy również uzyskać reprezentacje krawędzi oraz całych grafów. 

## 2.2. Zadania uczenia maszynowego na grafach

Zanim przejdziemy do omówienia konkretnych metod uczenia reprezentacji grafów, przygotujemy sobie narzędzia (funkcje) do ewaluacji otrzymanych wektorów reprezentacji trzech najpopularniejszych zadaniach związanych z przetwarzaniem grafów:

1. Klasyfikacja wierzchołków
2. Predykcja krawędzi
3. Klasyfikacja grafów

## 2.3. Klasyfikacja wierzchołków

Zakładamy, że każdy wierzchołek w grafie może zostać skojarzony z pewną klasą – np.:
- w sieci społecznej – płeć,
- w cząsteczkach chemicznych – rodzaj atomu). 

Korzystając z wektorów reprezentacji $\mathbf{Z}$, uczymy klasyfikator $f$ (np. liniowy lub sieć MLP) przewidywania klasy $c_u$ danego wierzchołka $u$, tzn. $f(z_u) = c_u$ (ang. **node classification**). Musimy podzielić zbiór wierzchołków na treningowe oraz testowe. Klasyfikator uczymy na wektorach reprezentacji i klasach wierzchołków treningowych, a metryki ewaluacyjne wyliczamy uwzględniając predykcje klasyfikatora na wektorach reprezentacji wierzchołków ze zbioru testowego. 

Często w grafach tylko niewielka część wierzchołków jest oznaczona (ma przypisane klasy).

**Uwaga:** W trakcie uczenia wektorów reprezentacji pomijamy informację o klasach wierzchołków (*unsupervised learning*).

### Zadanie 2.1. (1.5 pkt)
Zaimplementuj poniższą funkcję `evaluate_node_classification`, która zweryfikuje jakość podanych wektorów reprezentacji wierzchołków `z` w zadaniu klasyfikacji wierzchołków:

- podziel wierzchołki grafu na zbiór treningowy oraz testowy, uwzględniając rozmiar zbioru testowego `test_size`,
- zastosuj klasyfikator regresji logistycznej na wektorach reprezentacji oraz etykietach wierzchołków,
- oblicz metrykę AUC na zbiorze treningowym oraz testowym.


**Uwaga:** Pomijamy w tym zadaniu fakt, że wiele zbiorów ma zdefiniowany "odgórnie" podział wierzchołków - zobacz atrybuty `train_mask`, `val_mask` oraz `test_mask` w obiekcie `Data`.

In [41]:
from typing import Dict

import torch
import numpy as np
from torch_geometric.data import Data
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import label_binarize
from sklearn.metrics import roc_auc_score


def evaluate_node_classification(
    data: Data,
    z: torch.Tensor,
    test_size: float = 0.4,
) -> Dict[str, float]:
    y = data.y.numpy()
    train_mask, test_mask = train_test_split(range(len(y)), test_size=test_size, stratify=y)

    z_train = z[train_mask]
    z_test = z[test_mask]
    y_train = data.y[train_mask]
    y_test = data.y[test_mask]

    classifier = LogisticRegression(multi_class='ovr')
    classifier.fit(z_train, y_train)

    y_train_pred = classifier.predict_proba(z_train)
    y_test_pred = classifier.predict_proba(z_test)

    auc_train = roc_auc_score(y_train, y_train_pred, multi_class='ovr')
    auc_test = roc_auc_score(y_test, y_test_pred, multi_class='ovr')
    
    evaluation_results = {
        'train_auc': auc_train,
        'test_auc': auc_test
    }
    return evaluation_results

Zweryfikujmy, że funkcja działa na losowo zainicjalizowanej macierzy wektorów reprezentacji wierzchołków i zbiorze Cora:

In [47]:
def test_evaluate_node_classification():
    from torch_geometric.datasets import Planetoid

    torch.manual_seed(42)

    data = Planetoid(root="./data/", name="Cora")[0]
    z = torch.randn(data.num_nodes, 128)

    metrics = evaluate_node_classification(data=data, z=z)

    assert "train_auc" in metrics
    assert "test_auc" in  metrics

    print(metrics)
    
    
test_evaluate_node_classification()

{'train_auc': 0.7433846284967799, 'test_auc': 0.5187286084390209}


## 2.4. Predykcja krawędzi

W wielu przypadkach możemy założyć, że nie posiadamy pełnej informacji o strukturze grafu, tzn. nie obserwujemy w zbiorze wszystkich krawędzi, które w rzeczywistości istnieją w danym grafie. Możemy jednak wyuczyć model, który uzupełni brakujące krawędzie (ang. **link prediction**). 

Zbiór krawędzi grafu dzielimy na zbiór treningowy oraz testowy. Krawędzie ze zbioru testowego *usuwamy z grafu*, aby były nieobserwowane w trakcie uczenia wektorów reprezentacji. W niektórych wariantach tego zadania, staramy się również podzielić wierzchołki tak, aby krawędzie testowe występowały tylko między wybranym podzbiorem wierzchołków (które również na czas uczenia usuwamy) – ten scenariusz jednak nie będziemy stosować w ramach zajęć.

Na tym etapie możemy zauważyć pewnien problem – mamy tylko informację o istniejących w grafie krawędziach. Nie wiadomo jak wyuczyć model klasyfikatora? Potrzebowalibyśmy jeszcze informacji o krawędziach, które nie istnieją w grafie, wtedy możemy uznać istniejące krawędzie (przykłady pozytywne) jako klasę $1$, a nieistniejące (przykłady negatywne) jako klasę $0$. 

Dokładnie tak rozwiążemy nasz problem – każdej krawędzi w obecnym zbiorze treningowym i testowym przypisujemy klasę $1$. Następnie dla każdej krawędzi losujemy ze zbioru wszystkich możliwych krawędzi, taką która nie istnieje w danym grafie i przypisujemy jej klasę $0$ (ang. *balanced negative sampling*).

Wybór odpowiednich negatywnych przypadków jest bardziej skomplikowany i istnieje wiele strategii losowania, jednak pozostaniemy przy najprostszym scenariuszu losując krawędź zgodnie z rozkładem jednostajnym (tzn. każdy wybór tak samo prawdopodobny).

Kolejnym zagadnieniem jest jak otrzymać wektor reprezentacji dla krawędzi, skoro założyliśmy, że korzystamy z wektorów dla wierzchołków? Tutaj również mamy wiele możliwości, jednak najpopularniejszym rozwiązaniem jest wykorzystanie jednej z następujących transformacji wektorów reprezentacji wierzchołków $z_{uv} = z_u \circ z_v$, gdzie $\circ: \mathcal{V} \times \mathcal{V} \to \mathbb{R}^{d}$. Metody te zostały zaproponowane w pracy [Node2vec](https://arxiv.org/pdf/1607.00653.pdf) i są aplikowane na każdym elemencie wektora reprezentacji osobno (ang. *element-wise*):

| Nazwa | Wzór  |
|-------|-------|
| Średnia  | $$ z_{uv} = \frac{z_u + z_v}{2}$$ |
| Hadamard | $$ z_{uv} = z_u * z_v$$ |
| L1       | $$ z_{uv} = |z_u - z_v|$$ |
| L2       | $$ z_{uv} = |z_u - z_v|^2$$ |

Po przekształceniu wektorów reprezentacji wierzchołków na wektory krawędzi, możemy wyuczyć klasyfikator binarny na wektorach krawędzi ze zbioru treningowego (uwzględniając przypadki pozytywne i negatywne), a następnie wyliczyć wartości miar ewaluacyjnych w oparciu o predykcje klasyfikatora na wektorach krawędzi ze zbioru testowego.

### Zadanie 2.2 (5 pkt)
Zaimplementuj następujące funkcje:

- `prepare_train_test_sets` (2 pkt):
    - podaną listę krawędzi `edge_index` podziel na zbiór treningowy (`train_edges_pos`) oraz testowy (`test_edges_pos`) uwzględniając wielkość zbioru testowego `test_size`,
    - dla każdej krawędzi wylosuj przypadek negatywny: `train_edges_neg` oraz `test_edges_neg` (zobacz funkcję `negative_sampling` z biblioteki PyTorch-Geometric),
    - utwórz listę (tensor) krawędzi treningowych `train_edges` (konkatenacja `train_edges_pos` oraz `train_edges_neg`) o wymiarach `2 x łączna liczba krawędzi`,
    - utwórz taką samą listę dla krawędzi testowych `test_edges`,
    - utwórz wektory etykiet (zera i jedynki): `y_train` oraz `y_test`.
    
- `transform_to_edge_embeddings` (2 pkt):
    - dla podanej listy krawędzi `edge_index` oraz wektorów reprezentacji wierzchołków `z` obliczy reprezentacje krawędzi
    - argument `transformation_name` wyznacza metodę transformacji: "average", "hadamard", "L1" lub "L2" (zgodnie z powyższą tabelką)

- `evalute_link_prediction` (1 pkt):
    - dla podanych: listy krawędzi treningowych `train_edges` (wraz z `y_train`) oraz testowych `test_edges` (wraz z `y_test`), metody transformacji do reprezentacji krawędzi `transformation_name` oraz reprezentacji wierzchołków `z`, przeprowadzi ewaluację tych reprezentacji w zadaniu predykcji krawędzi
    - zastosuj klasyfikator regresji logistycznej
    - oblicz miary AUC dla zbioru treningowego i testowego

In [70]:
from typing import Dict
from torch_geometric.data import Data
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression
from torch_geometric.utils import negative_sampling
from sklearn.model_selection import train_test_split


def prepare_train_test_sets(
    edge_index: torch.Tensor,
    test_size: float = 0.4,
) -> Dict[str, torch.Tensor]:
    num_nodes = edge_index.max().item() + 1
    
    edge_index_T = edge_index.t()
    edges_pos_train, edges_pos_test = train_test_split(edge_index_T, test_size=test_size, random_state=42)
    train_edges_pos = edges_pos_train.t()
    test_edges_pos = edges_pos_test.t()

    train_edges_neg = negative_sampling(edge_index=train_edges_pos, num_nodes=num_nodes)
    test_edges_neg = negative_sampling(edge_index=test_edges_pos, num_nodes=num_nodes)

    train_edges = torch.cat([train_edges_pos, train_edges_neg], dim=1)
    test_edges = torch.cat([test_edges_pos, test_edges_neg], dim=1)
    
    y_train = torch.cat([torch.ones(train_edges_pos.size(1)), torch.zeros(train_edges_neg.size(1))], dim=0)
    y_test = torch.cat([torch.ones(test_edges_pos.size(1)), torch.zeros(test_edges_neg.size(1))], dim=0)

    return {
        "train_edges_pos": train_edges_pos,
        
        "train_edges": train_edges,
        "y_train": y_train,
        
        "test_edges": test_edges,
        "y_test": y_test,
    }
    
    
    
def test_prepare_train_test_sets():
    from torch_geometric.datasets import Planetoid
    
    data = Planetoid(root="./data/", name="Cora")[0]

    train_size = 0.6
    out = prepare_train_test_sets(
        edge_index=data.edge_index,
        test_size=1 - train_size,
    )

    num_train_edges_pos = int(train_size * data.num_edges)
    num_test_edges_pos = data.num_edges - num_train_edges_pos
    
    assert out["train_edges_pos"].shape[1] == num_train_edges_pos
    
    assert out["train_edges"].shape == (2, 2 * num_train_edges_pos)
    assert out["y_train"].shape == (2 * num_train_edges_pos,)
    
    assert out["test_edges"].shape == (2, 2 * num_test_edges_pos)
    assert out["y_test"].shape == (2 * num_test_edges_pos,)
    
    
test_prepare_train_test_sets()

In [64]:
def transform_to_edge_embeddings(
    edge_index: torch.Tensor,
    z: torch.Tensor,
    transformation_name: str,
) -> torch.Tensor:
    source_nodes_embeddings = z[edge_index[0, :]]
    target_nodes_embeddings = z[edge_index[1, :]]
    
    if transformation_name == "average":
        edge_embeddings = (source_nodes_embeddings + target_nodes_embeddings) / 2
    elif transformation_name == "hadamard":
        edge_embeddings = source_nodes_embeddings * target_nodes_embeddings
    elif transformation_name == "L1":
        edge_embeddings = torch.abs(source_nodes_embeddings - target_nodes_embeddings)
    elif transformation_name == "L2":
        edge_embeddings = (source_nodes_embeddings - target_nodes_embeddings) ** 2
    else:
        raise ValueError(f"Unknown transformation name: {transformation_name}")

    return edge_embeddings


def test_transform_to_edge_embeddings():
    ei = torch.tensor([[0], [1]])
    z = torch.tensor([
        [1, 2, 3],
        [4, 5, 6],
    ])
    
    assert (transform_to_edge_embeddings(ei, z, "average") == torch.tensor([[2.5, 3.5, 4.5]])).all()
    assert (transform_to_edge_embeddings(ei, z, "hadamard") == torch.tensor([[4, 10, 18]])).all()
    assert (transform_to_edge_embeddings(ei, z, "L1") == torch.tensor([[3, 3, 3]])).all()
    assert (transform_to_edge_embeddings(ei, z, "L2") == torch.tensor([[9, 9, 9]])).all()
    
    
test_transform_to_edge_embeddings()

In [78]:
def evaluate_link_prediction(
    train_edges: torch.Tensor,
    y_train: torch.Tensor,
    test_edges: torch.Tensor,
    y_test: torch.Tensor,
    transformation_name: str,
    z: torch.Tensor,
) -> Dict[str, float]:
    train_edges_emb = transform_to_edge_embeddings(train_edges, z, transformation_name)
    test_edges_emb = transform_to_edge_embeddings(test_edges, z, transformation_name)
    
    model = LogisticRegression()
    model.fit(train_edges_emb, y_train)
    
    y_train_pred = model.predict_proba(train_edges_emb)[:, 1]
    y_test_pred = model.predict_proba(test_edges_emb)[:, 1]
    
    auc_train = roc_auc_score(y_train, y_train_pred)
    auc_test = roc_auc_score(y_test, y_test_pred)

    return {
        'train_auc': auc_train,
        'test_auc': auc_test
    }
    
    
def test_evaluate_link_prediction():
    from torch_geometric.datasets import Planetoid
    
    data = Planetoid(root="./data/", name="Cora")[0]

    out = prepare_train_test_sets(edge_index=data.edge_index)
    
    z = torch.randn(data.num_nodes, 128)
    
    metrics = evaluate_link_prediction(
        train_edges=out["train_edges"],
        y_train=out["y_train"],
        test_edges=out["test_edges"],
        y_test=out["y_test"],
        transformation_name="average",
        z=z,
    )
    
    assert "train_auc" in metrics.keys()
    assert "test_auc" in metrics.keys()
    
    print(metrics)


test_evaluate_link_prediction()

{'train_auc': 0.6113308489222387, 'test_auc': 0.5770839065682785}


## 2.5. Klasyfikacja grafów

Zadanie to jest podobne do klasyfikacji wierzchołków, przy czym tutaj klasa jest przypisana do całego grafu. Przykładem może być klasyfikacja białek (reprezentowanych jako grafy). 

Podejście jest analogiczne jak w punkcie **2.3**. Zbiór grafów dzielimy na zbiór treningowy oraz testowy i uczymy odpowiedni klasyfikator. Jedynym zagadnieniem, które musimy rozwiązać jest sposób uzyskania wektora reprezentacji dla całego grafu na podstawie wektorów reprezentacji pojedynczych wierzchołków. Będziemy rozważać proste przekształcenia, które będą agregować wektory wierzchołków w jeden wektor opisujacy cały graf:

- uśrednianie: $z_\mathcal{G} = \frac{1}{|\mathcal{V}|} \sum_{u \in \mathcal{V}} z_u$
- redukcja max (ang. **max pooling**): $z_\mathcal{G} = \max_i \{z_1^{(i)}, z_2^{(i)}, \ldots, z_{|\mathcal{V}|}^{(i)} \} \;\forall i = 1 \ldots d$, gdzie $z_u^{(i)}$ oznacza $i$-ty element wektora $z_u$,
- redukcja min (ang. **min pooling**): $z_\mathcal{G} = \min_i \{z_1^{(i)}, z_2^{(i)}, \ldots, z_{|\mathcal{V}|}^{(i)} \} \;\forall i = 1 \ldots d$

Możemy również zastosować inne strategie wyznaczania wektora opisującego cały graf, ale więcej o tym na następnym wykładzie.

### Zadanie 2.3. (3.5 pkt)
Zaimplementuj funkcje:

- `transform_to_graph_embedding` (1.5 pkt):
    - dla podanej macierzy reprezentacji wierzchołków `z` wyznaczy wektor reprezentacji całego grafu
    - argument `transformation_name` wyznacza metodę transformacji: "average", "max_pooling", "min_pooling"
    
- `evaluate_graph_classification` (2 pkt):
    - dla podanej listy macierzy reprezentacji grafów `z`, wektora etykiet grafów `y` oraz metody transformacji `transformation_name`, przeprowadzi ewaluację reprezentacji w zadaniu klasyfikacji grafów
    - podziel grafy na zbiór treningowy oraz testowy, gdzie wielkość zbioru testowego jest określona za pomocą parametru `test_size`,
    - przetransformuj listę macierzy reprezentacji wierzchołków w jedną macierz reprezentacji grafów,
    - zastosuj klasyfikator regresji logistycznej
    - oblicz miary AUC dla zbioru treningowego i testowego

In [80]:
def transform_to_graph_embedding(
    z: torch.Tensor,
    transformation_name: str,
) -> torch.Tensor:
    if transformation_name == "average":
        return z.mean(dim=0)
    elif transformation_name == "max_pooling":
        return z.max(dim=0)[0]
    elif transformation_name == "min_pooling":
        return z.min(dim=0)[0]
    else:
        raise ValueError(f"Unknown transformation: {transformation_name}")

        

def test_transform_to_graph_embedding():
    z = torch.tensor([
        [1, 2, 3],
        [4, 5, 6],
        [7, 8, 9],
    ])
    
    assert (transform_to_graph_embedding(z, "average") == torch.tensor([4, 5, 6])).all()
    assert (transform_to_graph_embedding(z, "max_pooling") == torch.tensor([7, 8, 9])).all()
    assert (transform_to_graph_embedding(z, "min_pooling") == torch.tensor([1, 2, 3])).all()

In [88]:
from typing import List


def evaluate_graph_classification(
    z: List[torch.Tensor],
    y: torch.Tensor,
    transformation_name: str,
    test_size: float = 0.4,
) -> Dict[str, float]:
    z_graphs = [transform_to_graph_embedding(z_i, transformation_name) for z_i in z]
    z_graphs = torch.stack(z_graphs)
    
    z_train, z_test, y_train, y_test = train_test_split(z_graphs, y, test_size=test_size, random_state=42)
    classifier = LogisticRegression(multi_class='ovr')
    classifier.fit(z_train, y_train)
    
    y_train_pred = classifier.predict_proba(z_train)
    y_test_pred = classifier.predict_proba(z_test)
    
    auc_train = roc_auc_score(y_train, y_train_pred, multi_class="ovr")
    auc_test = roc_auc_score(y_test, y_test_pred, multi_class="ovr")
    
    return {
        'train_auc': auc_train,
        'test_auc': auc_test
    }
    
    
def test_evaluate_graph_classification():
    from torch_geometric.datasets import TUDataset
    
    enzymes = TUDataset(root="./data", name="ENZYMES")
    
    y = torch.tensor([e.y for e in enzymes])
    z = [torch.randn(e.num_nodes, 128) for e in enzymes]

    metrics = evaluate_graph_classification(z, y, "average")
    
    assert "train_auc" in metrics.keys()
    assert "test_auc" in metrics.keys()
    
    print(metrics)
    
    
test_evaluate_graph_classification()

{'train_auc': 0.9237988978654822, 'test_auc': 0.5085761696598391}
